In [2]:
%pwd

'/Users/yasith/Documents/projects/medChat/research'

In [3]:
import os
os.chdir("../")

In [5]:
%pwd

'/Users/yasith/Documents/projects/medChat'

## importing libraries

In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

## Extracting Data from the pdf

In [7]:
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    
    documents=loader.load()

    return documents

In [8]:
extracted_data = load_pdf_file(data='Data/')